# 🎨 Manhwa Translator v2

Batch processing pipeline for translating Korean manhwa CBZ files to English.

**Pipeline:**
1. Upload ZIP containing CBZ files
2. Extract each CBZ file
3. Batch process images (16 at a time)
4. Detect bubbles → Extract text → Translate → Render
5. Repack processed images back to CBZ
6. Download all translated CBZ files as ZIP

## 1. Install Dependencies

In [ ]:
!pip install ultralytics easyocr transformers accelerate opencv-python pillow sentencepiece -q

## 2. Download YOLO Bubble Detector Model

In [ ]:
!wget https://huggingface.co/ogkalu/comic-speech-bubble-detector-yolov8m/resolve/main/comic-speech-bubble-detector.pt -O bubble_detector.pt

## 3. Upload ZIP File (containing CBZ files)

In [ ]:
import os
import zipfile
import shutil
from google.colab import files

# Create directories
BASE_DIR = 'manhwa_work'
INPUT_DIR = os.path.join(BASE_DIR, 'input')
OUTPUT_DIR = os.path.join(BASE_DIR, 'output')
TEMP_DIR = os.path.join(BASE_DIR, 'temp')

for d in [INPUT_DIR, OUTPUT_DIR, TEMP_DIR]:
    os.makedirs(d, exist_ok=True)

# Upload ZIP file
print("Please upload your ZIP file containing CBZ files:")
uploaded = files.upload()

# Extract ZIP
zip_filename = list(uploaded.keys())[0]
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(INPUT_DIR)

# Find all CBZ files
cbz_files = []
for root, dirs, files_list in os.walk(INPUT_DIR):
    for f in files_list:
        if f.lower().endswith('.cbz'):
            cbz_files.append(os.path.join(root, f))

cbz_files.sort()
print(f"\n✅ Found {len(cbz_files)} CBZ files:")
for cbz in cbz_files:
    print(f"  - {os.path.basename(cbz)}")

## 4. Load Models

In [ ]:
import torch
from ultralytics import YOLO
import easyocr
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import cv2

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load YOLO bubble detector
print("Loading YOLO bubble detector...")
bubble_model = YOLO('bubble_detector.pt')

# Load EasyOCR reader for Korean
print("Loading EasyOCR (Korean)...")
ocr_reader = easyocr.Reader(['ko'], gpu=torch.cuda.is_available())

# Load NLLB translation model (fine-tuned for Korean to English)
print("Loading NLLB fine-tuned Ko2En translation model...")
nllb_model_name = "NHNDQ/nllb-finetuned-ko2en"
tokenizer = AutoTokenizer.from_pretrained(nllb_model_name)
translation_model = AutoModelForSeq2SeqLM.from_pretrained(nllb_model_name).to(device)

print("Using OpenCV inpainting...")
print("\n✅ All models loaded successfully!")

## 5. Define Processing Functions

In [ ]:
import cv2
import numpy as np
import textwrap
from PIL import Image, ImageDraw, ImageFont
from concurrent.futures import ThreadPoolExecutor

# Font path for Colab
FONT_PATH = "/usr/share/fonts/truetype/liberation/LiberationSans-Bold.ttf"
try:
    ImageFont.truetype(FONT_PATH, 12)
except:
    FONT_PATH = "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf"

def translate_text(korean_text):
    """Translate Korean text to English using fine-tuned NLLB."""
    if not korean_text:
        return ""
    
    inputs = tokenizer(korean_text, return_tensors="pt", padding=True, truncation=True, max_length=256).to(device)
    
    translated_tokens = translation_model.generate(
        **inputs,
        max_length=256,
        num_beams=5,
        early_stopping=True
    )
    
    translated_text = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
    return translated_text

def translate_batch(texts):
    """Translate a batch of Korean texts to English."""
    if not texts:
        return []
    
    # Filter empty texts
    valid_indices = [i for i, t in enumerate(texts) if t.strip()]
    valid_texts = [texts[i] for i in valid_indices]
    
    if not valid_texts:
        return [""] * len(texts)
    
    inputs = tokenizer(valid_texts, return_tensors="pt", padding=True, truncation=True, max_length=256).to(device)
    
    translated_tokens = translation_model.generate(
        **inputs,
        max_length=256,
        num_beams=5,
        early_stopping=True
    )
    
    translations = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)
    
    # Map back to original indices
    result = [""] * len(texts)
    for i, idx in enumerate(valid_indices):
        result[idx] = translations[i]
    
    return result

def process_single_image(image_path):
    """Process a single image: detect bubbles, extract text, translate, white-out and render."""
    img = cv2.imread(image_path)
    if img is None:
        return None, []
    
    # Detect bubbles
    yolo_result = bubble_model(image_path, conf=0.5, verbose=False)
    
    # Convert to PIL for drawing
    pil_img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(pil_img)
    
    bubble_data = []
    
    # Process each detected bubble - extract text
    for bubble in yolo_result[0].boxes:
        bx1, by1, bx2, by2 = map(int, bubble.xyxy[0])
        
        bubble_crop = img[by1:by2, bx1:bx2]
        ocr_res = ocr_reader.readtext(bubble_crop)
        
        if not ocr_res:
            continue
        
        korean_text = ' '.join([res[1] for res in ocr_res])
        
        # Store bubble info for batch translation
        bubble_data.append({
            'box': (bx1, by1, bx2, by2),
            'korean_text': korean_text,
            'ocr_res': ocr_res
        })
    
    return (img, pil_img, draw, bubble_data), [b['korean_text'] for b in bubble_data]

def render_translated_image(img_data, translations):
    """Render translated text onto image."""
    if img_data is None:
        return None
    
    img, pil_img, draw, bubble_data = img_data
    
    for i, data in enumerate(bubble_data):
        bx1, by1, bx2, by2 = data['box']
        english_text = translations[i] if i < len(translations) else ""
        
        # WHITE-OUT: Draw white rectangles over Korean text
        for res in data['ocr_res']:
            poly = np.array(res[0], dtype=np.int32)
            x_min = poly[:, 0].min() + bx1
            y_min = poly[:, 1].min() + by1
            x_max = poly[:, 0].max() + bx1
            y_max = poly[:, 1].max() + by1
            draw.rectangle([x_min-2, y_min-2, x_max+2, y_max+2], fill="white")
        
        if not english_text:
            continue
        
        # Render English text
        padding = 10
        target_w = (bx2 - bx1) - (2 * padding)
        target_h = (by2 - by1) - (2 * padding)
        
        font_size = int(img.shape[0] / 50)
        font = ImageFont.truetype(FONT_PATH, font_size)
        
        while font_size > 8:
            font = ImageFont.truetype(FONT_PATH, font_size)
            avg_char_w = font.getlength("a")
            chars_per_line = max(1, int(target_w / avg_char_w))
            wrapped = textwrap.wrap(english_text, width=chars_per_line, break_long_words=False)
            total_text_h = len(wrapped) * (font_size + 4)
            if total_text_h <= target_h:
                break
            font_size -= 2
        
        current_y = by1 + (target_h - total_text_h) // 2 + padding
        for line in wrapped:
            bbox_line = draw.textbbox((0, 0), line, font=font)
            line_w = bbox_line[2] - bbox_line[0]
            current_x = bx1 + (target_w - line_w) // 2 + padding
            draw.text((current_x, current_y), line, font=font, fill="black")
            current_y += font_size + 4
    
    return pil_img

def process_batch(image_paths, output_dir):
    """Process a batch of images."""
    # Step 1: Process all images (YOLO + OCR)
    all_img_data = []
    all_texts = []
    text_counts = []
    
    for img_path in image_paths:
        img_data, texts = process_single_image(img_path)
        all_img_data.append((img_path, img_data))
        all_texts.extend(texts)
        text_counts.append(len(texts))
    
    # Step 2: Batch translate all texts
    if all_texts:
        all_translations = translate_batch(all_texts)
    else:
        all_translations = []
    
    # Step 3: Render and save
    translation_idx = 0
    for i, (img_path, img_data) in enumerate(all_img_data):
        count = text_counts[i]
        translations = all_translations[translation_idx:translation_idx + count]
        translation_idx += count
        
        if img_data is None or img_data[0] is None:
            # Copy original if processing failed
            src = img_path
            dst = os.path.join(output_dir, os.path.basename(img_path))
            shutil.copy2(src, dst)
            continue
        
        result_img = render_translated_image(img_data, translations)
        if result_img:
            output_path = os.path.join(output_dir, os.path.basename(img_path))
            result_img.save(output_path)
        else:
            shutil.copy2(img_path, os.path.join(output_dir, os.path.basename(img_path)))

## 6. Process All CBZ Files

In [ ]:
from tqdm import tqdm
import math

BATCH_SIZE = 16

print(f"Processing {len(cbz_files)} CBZ files...\n")

for cbz_path in cbz_files:
    cbz_name = os.path.basename(cbz_path)
    print(f"\n{'='*60}")
    print(f"📚 Processing: {cbz_name}")
    
    # Create temp directories for this CBZ
    cbz_temp_input = os.path.join(TEMP_DIR, 'cbz_input')
    cbz_temp_output = os.path.join(TEMP_DIR, 'cbz_output')
    
    # Clean and create temp dirs
    for d in [cbz_temp_input, cbz_temp_output]:
        if os.path.exists(d):
            shutil.rmtree(d)
        os.makedirs(d)
    
    # Extract CBZ (it's just a ZIP file)
    try:
        with zipfile.ZipFile(cbz_path, 'r') as zip_ref:
            zip_ref.extractall(cbz_temp_input)
    except Exception as e:
        print(f"  ⚠️ Error extracting CBZ: {e}")
        continue
    
    # Get all files from extracted CBZ
    image_extensions = ('.png', '.jpg', '.jpeg', '.webp', '.gif')
    image_files = []
    non_image_files = []
    
    for root, dirs, files_list in os.walk(cbz_temp_input):
        for f in files_list:
            if f.startswith('.'):
                continue
            full_path = os.path.join(root, f)
            rel_path = os.path.relpath(full_path, cbz_temp_input)
            
            if f.lower().endswith(image_extensions):
                image_files.append(full_path)
            else:
                # Copy non-image files directly to output (preserve metadata like ComicInfo.xml)
                non_image_files.append((full_path, rel_path))
    
    image_files.sort()
    print(f"  Found {len(image_files)} images, {len(non_image_files)} other files")
    
    # Copy non-image files to output directory (preserving folder structure)
    for src_path, rel_path in non_image_files:
        dst_path = os.path.join(cbz_temp_output, rel_path)
        os.makedirs(os.path.dirname(dst_path), exist_ok=True)
        shutil.copy2(src_path, dst_path)
    
    if not image_files:
        print(f"  ⚠️ No images found, skipping...")
        continue
    
    # Process in batches
    num_batches = math.ceil(len(image_files) / BATCH_SIZE)
    
    for batch_idx in tqdm(range(num_batches), desc=f"  Batches"):
        start_idx = batch_idx * BATCH_SIZE
        end_idx = min(start_idx + BATCH_SIZE, len(image_files))
        batch_images = image_files[start_idx:end_idx]
        
        process_batch(batch_images, cbz_temp_output)
    
    # Repack to CBZ
    output_cbz_path = os.path.join(OUTPUT_DIR, cbz_name)
    
    with zipfile.ZipFile(output_cbz_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files_list in os.walk(cbz_temp_output):
            for f in sorted(files_list):
                file_path = os.path.join(root, f)
                arcname = os.path.relpath(file_path, cbz_temp_output)
                zipf.write(file_path, arcname)
    
    print(f"  ✅ Saved: {cbz_name}")

# Clean up temp directory
shutil.rmtree(TEMP_DIR, ignore_errors=True)

print(f"\n{'='*60}")
print("✅ All CBZ files processed!")

## 7. Create & Download Output ZIP

In [ ]:
from google.colab import files

# Create output ZIP
output_zip = 'translated_manhwa.zip'

with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files_list in os.walk(OUTPUT_DIR):
        for f in files_list:
            if f.lower().endswith('.cbz'):
                file_path = os.path.join(root, f)
                arcname = os.path.relpath(file_path, OUTPUT_DIR)
                zipf.write(file_path, arcname)

# Count CBZ files in output
output_cbz_count = len([f for f in os.listdir(OUTPUT_DIR) if f.lower().endswith('.cbz')])
print(f"✅ Created {output_zip} with {output_cbz_count} translated CBZ files")

# Download
print("Starting download...")
files.download(output_zip)

## 8. Cleanup Storage (Optional)

Run this cell to remove all uploaded and processed files to free up Colab storage.

In [ ]:
# import shutil
# import os

# # Remove work directories
# if os.path.exists(BASE_DIR):
#     shutil.rmtree(BASE_DIR)
#     print(f"✅ Removed {BASE_DIR}/")

# # Remove uploaded ZIP
# if 'zip_filename' in dir() and os.path.exists(zip_filename):
#     os.remove(zip_filename)
#     print(f"✅ Removed {zip_filename}")

# # Remove output ZIP
# if os.path.exists('translated_manhwa.zip'):
#     os.remove('translated_manhwa.zip')
#     print("✅ Removed translated_manhwa.zip")

# # Remove YOLO model
# if os.path.exists('bubble_detector.pt'):
#     os.remove('bubble_detector.pt')
#     print("✅ Removed bubble_detector.pt")

# print("\n🧹 Cleanup complete!")